In [9]:
#1

def deco(func):
    def inner():
        print("running inner()")
    return inner

@deco
def target():
    print("running target()")

target()

target


running inner()


<function __main__.deco.<locals>.inner()>

In [10]:
#2

registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')

def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

if __name__ == '__main__':
    main()
    

running register(<function f1 at 0x000002817B8EEA20>)
running register(<function f2 at 0x000002817BEF0180>)
running main()
registry -> [<function f1 at 0x000002817B8EEA20>, <function f2 at 0x000002817BEF0180>]
running f1()
running f2()
running f3()


In [11]:
#3

promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

@promotion
def large_order(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

def best_promo(order):
    return max(promo(order) for promo in promos)


In [12]:
#4

def f1(a):
    print(a)
    print(b)

#b = 6
f1(3)


3


NameError: name 'b' is not defined

In [ ]:
#5

b = 6

def f2(a):
    print(a)
    print(b)
    b = 9

print(f2(3))


3


UnboundLocalError: cannot access local variable 'b' where it is not associated with a value

In [21]:
#6

b = 6

def f3(a):
    global b
    print(a)
    print(b)
    b = 9

f3(3)

print(b)

b = 30
f3(3)


3
6
9
3
30


In [20]:
#7

class Averager():
    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

avg = Averager()
print(avg(10))

print(avg(11))

print(avg(12))


10.0
10.5
11.0


In [19]:
#8

def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)

    return averager

avg = make_averager()

print(avg(10))

print(avg(11))

print(avg(12))


10.0
10.5
11.0


In [18]:
#9

print(avg.__code__.co_varnames)

print(avg.__code__.co_freevars)

print(avg.__closure__)

print(avg.__closure__[0].cell_contents)


('new_value',)
('count', 'total')
(<cell at 0x000002817BA7B220: int object at 0x00007FFF537419B8>, <cell at 0x000002817BA78E80: int object at 0x00007FFF53741AD8>)
1


In [13]:
#10

def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        count += 1
        total += new_value
        return total/count

    return averager


In [14]:
#11

def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total/count

    return averager

avg = make_averager()

print(avg(10))


10.0


In [15]:
#12

import time

def clock(func):
    def clocked(*args):
        t0 = time.time()
        result = func(*args)
        elapsed = time.time() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked


In [16]:
#13

@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

    print(factorial.__name__)
    

**************************************** Calling snooze(.123)
[0.12390447s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000000s] factorial(1) -> 1
[0.00000000s] factorial(2) -> 2
[0.00000000s] factorial(3) -> 6
[0.00000000s] factorial(4) -> 24
[0.00000000s] factorial(5) -> 120
[0.00000000s] factorial(6) -> 720
6! = 720
clocked


In [17]:
#14

import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

    print(factorial.__name__)

**************************************** Calling snooze(.123)
[0.12381124s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000000s] factorial(1) -> 1
[0.00000000s] factorial(2) -> 2
[0.00000000s] factorial(3) -> 6
[0.00000000s] factorial(4) -> 24
[0.00000000s] factorial(5) -> 120
[0.00000000s] factorial(6) -> 720
6! = 720
factorial
